In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"
}  # Cabeçalho padrão
SEARCH_VALUE = "smartphone"  # Parâmetro de pesquisa

In [ ]:
# Requisição para página usando o SEARCH_VALUE de pesquisa
session = requests.Session()
response = session.get("https://www.mercadolivre.com.br/", timeout=5, headers=HEADERS)
session.headers.update
response = session.get(
    f"https://lista.mercadolivre.com.br/{SEARCH_VALUE}#D[A:{SEARCH_VALUE}]", timeout=5
)

In [ ]:
endereco_paginas = []
soup = BeautifulSoup(response.text, "html.parser")
nav_pagina = soup.findAll(class_="andes-pagination__link")

# TODO Adicionar lógica para acrescentar todas as novas páginas encontradas após uma navegação e evitar duplicidade de endereços
for pagina in nav_pagina:
    endereco_pagina = pagina.get("href")
    if isinstance(endereco_pagina, str):
        endereco_paginas.append(endereco_pagina)

In [ ]:
nomes_produtos = []
precos_produtos = []
avaliacoes_produtos = []
enderecos_requisitados = set()

for pagina in endereco_paginas:
    if pagina not in enderecos_requisitados:
        # verifica se a URL a ser requisitada já fora requisitada anteriormente
        response = session.get(pagina, timeout=5, headers=HEADERS)
        if response.status_code:
            print(f"URL requisitada: {pagina}")
            enderecos_requisitados.add(pagina)
            # Coleta das informações (nome do produto, preço, avalição (nota e quantidade) e navegação de página)
            soup = BeautifulSoup(response.text, "html.parser")
            nome_produto = soup.findAll(class_="poly-box poly-component__title")
            preco_produto = soup.findAll(class_="andes-money-amount__fraction")
            avaliacao_produto = soup.findAll(class_="andes-visually-hidden")
            nav_pagina = soup.findAll(class_="andes-pagination__link")
            # Adição das informações (nome do produto, preço, avaliação (nota e quantidade))
            for nome in nome_produto:
                nomes_produtos.append(nome.text)
            for preco in preco_produto:
                precos_produtos.append(preco.text)
            for avaliacao in avaliacao_produto:
                avaliacoes_produtos.append(avaliacao.text)
        else:
            print(f"Bad Requests, status code: {response.status_code}")
    elif pagina in enderecos_requisitados:
        print(f"Endereço já requisitado anteriormente: {pagina}")

In [ ]:
print(nomes_produtos)
print("\n")
print(precos_produtos)
print("\n")
print(avaliacoes_produtos)
print("\n")

In [ ]:
# Encerramento da conexão
session.close()